In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
!pip install -q scipy


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
model_id = 'EleutherAI/gpt-neo-125m'

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/gpt-neo-125m"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

In [5]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [6]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [7]:
from datasets import load_dataset

data = load_dataset("thanhnew2001/pokemon")
print(data)
data = data.map(lambda samples: tokenizer(samples["question"]), batched=True)

data = data.map(
    lambda samples: {
        "input_text": samples["question"] + '->:' + samples["answer"] + ''
    }
, remove_columns= ['question', 'answer'])

data = data.map(lambda examples: tokenizer(examples["input_text"]), batched=True)


print(data['train'])
data['train'][0:5]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['answer', 'question'],
        num_rows: 1405
    })
})


Map:   0%|          | 0/1405 [00:00<?, ? examples/s]

Map:   0%|          | 0/1405 [00:00<?, ? examples/s]

Map:   0%|          | 0/1405 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'input_text'],
    num_rows: 1405
})


{'input_ids': [[32,
   72,
   300,
   24247,
   299,
   71,
   22940,
   77,
   410,
   157,
   118,
   255,
   83,
   34754,
   239,
   157,
   119,
   241,
   782,
   289,
   24247,
   77,
   71,
   269,
   127,
   117,
   782,
   7844,
   509,
   7569,
   388,
   491,
   506,
   18912,
   157,
   119,
   247,
   66,
   289,
   24247,
   77,
   71,
   491,
   127,
   105,
   77,
   71,
   9887,
   269,
   157,
   119,
   100,
   64,
   281,
   71,
   28053,
   118,
   98,
   88,
   30,
   3784,
   25,
   45,
   71,
   22940,
   77,
   410,
   157,
   118,
   255,
   83,
   34754,
   239,
   157,
   119,
   241,
   782,
   289,
   24247,
   77,
   71,
   269,
   127,
   117,
   782,
   7844,
   509,
   7569,
   388,
   491,
   506,
   18912,
   157,
   119,
   247,
   66,
   289,
   24247,
   77,
   71,
   491,
   127,
   105,
   77,
   71,
   9887,
   269,
   157,
   119,
   100,
   64,
   281,
   71,
   28053,
   118,
   98,
   88,
   300,
   24247,
   38755,
   13],
  [33481,
   12

In [14]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=16,
        warmup_steps=2,
        max_steps=100,
        learning_rate=2e-6,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Step,Training Loss
1,0.000000
2,0.000000
3,0.000000
4,0.000000
5,0.000000
6,0.000000
7,0.000000
8,0.000000
9,0.000000
10,0.000000


TrainOutput(global_step=100, training_loss=0.0, metrics={'train_runtime': 121.8611, 'train_samples_per_second': 52.519, 'train_steps_per_second': 0.821, 'total_flos': 92227345661952.0, 'train_loss': 0.0, 'epoch': 4.55})

In [15]:
trainer.save_model('adapter-model')


In [16]:
from peft import PeftModel    

base_model = AutoModelForCausalLM.from_pretrained(
        'EleutherAI/gpt-neo-125m',
        return_dict=True,
        torch_dtype=torch.float16 
)

model = PeftModel.from_pretrained(base_model, 'adapter-model')
print(f"Running merge_and_unload")
model.to('cuda')
model = model.merge_and_unload() 
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neo-125m')

model.save_pretrained("merged")
tokenizer.save_pretrained("merged")


Running merge_and_unload


('merged/tokenizer_config.json',
 'merged/special_tokens_map.json',
 'merged/vocab.json',
 'merged/merges.txt',
 'merged/added_tokens.json',
 'merged/tokenizer.json')

In [17]:
model_id = "merged"

model = AutoModelForCausalLM.from_pretrained(model_id, load_in_4bit=True, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)




In [25]:
text = " Ai là nhân vật đồng hành cùng Ash Ketchum trong cuộc hành trình Pokémon của anh ấy?->"
inputs = tokenizer(text, return_tensors="pt").to("cuda:0")
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 Ai là nhân vật đồng hành cùng Ash Ketchum trong cuộc hành trình Pokémon của anh ấy?->!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [21]:
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
)

model_nf4 = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=nf4_config)

In [22]:


outputs = model_nf4.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 capital of norway!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [23]:
double_quant_nf4_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_use_double_quant=True
)

model_double = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=double_quant_nf4_config)
outputs = model_double.generate(**inputs, max_new_tokens=200)
print(outputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[3139,  286, 4249, 1014,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,  